# Naive Bayes Text Classification

## NLTK Overview

Natural Language ToolKit (NLTK)
```
   $ sudo pip install -U nltk
```

In [201]:
import nltk
from pandas import DataFrame

### Where to get data?

In [202]:
from nltk import download, download_shell, download_gui

In [203]:
# try it at home
download()

showing info http://www.nltk.org/nltk_data/


True

### Corpus

In [204]:
from nltk.corpus import brown, reuters

In [205]:
print brown.readme()

BROWN CORPUS

A Standard Corpus of Present-Day Edited American
English, for use with Digital Computers.

by W. N. Francis and H. Kucera (1964)
Department of Linguistics, Brown University
Providence, Rhode Island, USA

Revised 1971, Revised and Amplified 1979

http://www.hit.uib.no/icame/brown/bcm.html

Distributed with the permission of the copyright holder,
redistribution permitted.



In [206]:
print reuters.readme()


      The Reuters-21578 benchmark corpus, ApteMod version

This is a publically available version of the well-known Reuters-21578
"ApteMod" corpus for text categorization.  It has been used in
publications like these:

 * Yiming Yang and X. Liu. "A re-examination of text categorization
   methods".  1999.  Proceedings of 22nd Annual International SIGIR.
   http://citeseer.nj.nec.com/yang99reexamination.html

 * Thorsten Joachims. "Text categorization with support vector
   machines: learning with many relevant features".  1998. Proceedings
   of ECML-98, 10th European Conference on Machine Learning.
   http://citeseer.nj.nec.com/joachims98text.html

ApteMod is a collection of 10,788 documents from the Reuters financial
newswire service, partitioned into a training set with 7769 documents
and a test set with 3019 documents.  The total size of the corpus is
about 43 MB.  It is also available for download from
http://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html ,
which includ

In [207]:
print "\n".join(brown.categories())
print
print len(brown.categories())

adventure
belles_lettres
editorial
fiction
government
hobbies
humor
learned
lore
mystery
news
religion
reviews
romance
science_fiction

15


In [208]:
len(reuters.categories())

90

### Words interface

In [209]:
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

In [210]:
from itertools import islice
print " ".join(islice(brown.words(), 100)) + " ..."

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan ...


### Sents interface

In [211]:
brown.sents()

[[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.'], [u'The', u'jury', u'further', u'said', u'in', u'term-end', u'presentments', u'that', u'the', u'City', u'Executive', u'Committee', u',', u'which', u'had', u'over-all', u'charge', u'of', u'the', u'election', u',', u'``', u'deserves', u'the', u'praise', u'and', u'thanks', u'of', u'the', u'City', u'of', u'Atlanta', u"''", u'for', u'the', u'manner', u'in', u'which', u'the', u'election', u'was', u'conducted', u'.'], ...]

In [212]:
for sentence in islice(brown.sents(), 5):
    print " ".join(sentence)
    print

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .

The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .

The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .

`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .

The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' .



### Raw data

In [213]:
print brown.raw()[:1000]



	The/at Fulton/np-tl County/nn-tl Grand/jj-tl Jury/nn-tl said/vbd Friday/nr an/at investigation/nn of/in Atlanta's/np$ recent/jj primary/nn election/nn produced/vbd ``/`` no/at evidence/nn ''/'' that/cs any/dti irregularities/nns took/vbd place/nn ./.


	The/at jury/nn further/rbr said/vbd in/in term-end/nn presentments/nns that/cs the/at City/nn-tl Executive/jj-tl Committee/nn-tl ,/, which/wdt had/hvd over-all/jj charge/nn of/in the/at election/nn ,/, ``/`` deserves/vbz the/at praise/nn and/cc thanks/nns of/in the/at City/nn-tl of/in-tl Atlanta/np-tl ''/'' for/in the/at manner/nn in/in which/wdt the/at election/nn was/bedz conducted/vbn ./.


	The/at September-October/np term/nn jury/nn had/hvd been/ben charged/vbn by/in Fulton/np-tl Superior/jj-tl Court/nn-tl Judge/nn-tl Durwood/np Pye/np to/to investigate/vb reports/nns of/in possible/jj ``/`` irregularities/nns ''/'' in/in the/at hard-fought/jj primary/nn which/wdt was/bedz won/vbn by/in Mayor-nominate/nn-tl Ivan/np Allen/np Jr./

## Naive Bayes

Student Question: Formula?

In [214]:
unused = """
A classifier based on the Naive Bayes algorithm.  In order to find the
probability for a label, this algorithm first uses the Bayes rule to
express P(label|features) in terms of P(label) and P(features|label):

|                       P(label) * P(features|label)
|  P(label|features) = ------------------------------
|                              P(features)

The algorithm then makes the 'naive' assumption that all features are
independent, given the label:

|                       P(label) * P(f1|label) * ... * P(fn|label)
|  P(label|features) = --------------------------------------------
|                                         P(features)

Rather than computing P(featues) explicitly, the algorithm just
calculates the denominator for each label, and normalizes them so they
sum to one:

|                       P(label) * P(f1|label) * ... * P(fn|label)
|  P(label|features) = --------------------------------------------
|                        SUM[l]( P(l) * P(f1|l) * ... * P(fn|l) )
"""

In [215]:
from nltk.classify import NaiveBayesClassifier

In [216]:
from nltk.corpus import names

In [217]:
print names.readme()

Names Corpus, Version 1.3 (1994-03-29)
Copyright (C) 1991 Mark Kantrowitz
Additions by Bill Ross

This corpus contains 5001 female names and 2943 male names, sorted
alphabetically, one per line.

You may use the lists of names for any purpose, so long as credit is
given in any published work. You may also redistribute the list if you
provide the recipients with a copy of this README file. The lists are
not in the public domain (I retain the copyright on the lists) but are
freely redistributable.  If you have any additions to the lists of
names, I would appreciate receiving them.

Mark Kantrowitz <mkant+@cs.cmu.edu>
http://www-2.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/


In [218]:
names.fileids()

[u'female.txt', u'male.txt']

In [219]:
print names.words(fileids=['male.txt'])[:10]
print names.words(fileids=['female.txt'])[:10]

[u'Aamir', u'Aaron', u'Abbey', u'Abbie', u'Abbot', u'Abbott', u'Abby', u'Abdel', u'Abdul', u'Abdulkarim']
[u'Abagael', u'Abagail', u'Abbe', u'Abbey', u'Abbi', u'Abbie', u'Abby', u'Abigael', u'Abigail', u'Abigale']


### NLTK is smart =)

In [220]:
print names.words('male.txt')[:10]

[u'Aamir', u'Aaron', u'Abbey', u'Abbie', u'Abbot', u'Abbott', u'Abby', u'Abdel', u'Abdul', u'Abdulkarim']


In [221]:
from IPython.display import Image
Image(url="http://www.nltk.org/images/supervised-classification.png")

In [222]:
from random import shuffle

In [223]:
names = (
    [(name, 'male') for name in names.words('male.txt')] +
    [(name, 'female') for name in names.words('female.txt')]
)
shuffle(names)

In [224]:
names[:10]

[(u'Luella', 'female'),
 (u'Arlee', 'female'),
 (u'Bree', 'female'),
 (u'Sibylla', 'female'),
 (u'Gretal', 'female'),
 (u'Wenda', 'female'),
 (u'Jazmin', 'female'),
 (u'Curt', 'male'),
 (u'Westley', 'male'),
 (u'Amie', 'female')]

In [225]:
def make_features(text):
    return {
        'last_letter' : text[-1],
    }

In [234]:
featuresets = list((make_features(name), label) for name, label in names)

In [235]:
len(featuresets)

7944

In [236]:
test_size = 500
train_set, test_set = featuresets[test_size:], featuresets[:test_size]
classifier = NaiveBayesClassifier.train(train_set)

In [237]:
classifier.labels()

['male', 'female']

In [238]:
print classifier.classify(make_features('Mihkael'))
print classifier.classify(make_features('Elizabeth'))

male
female


In [239]:
from nltk.classify import accuracy

In [240]:
accuracy(classifier, test_set)

0.778

In [241]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = u'a'           female : male   =     37.4 : 1.0
             last_letter = u'k'             male : female =     31.7 : 1.0
             last_letter = u'f'             male : female =     16.6 : 1.0
             last_letter = u'p'             male : female =     11.9 : 1.0
             last_letter = u'v'             male : female =     11.2 : 1.0


In [242]:
prob_dist = classifier.prob_classify(make_features("Mikhael"))

In [243]:
prob_dist.__dict__

{'_log': True,
 '_prob_dict': {'female': -1.0239484849158842, 'male': -0.97644257307679}}

Studnet Question: Why log?

## Contest

In [ ]:
classifier.show_most_informative_features(10)

In [ ]:
from pandas import DataFrame
from string import letters
from nltk import bigrams, trigrams

In [244]:
def line(name):
    features = {}
    features['last'] = name[-1]
    return features

baisline("Xaronkn")

{'fist': 'X', 'last': 'n'}

In [245]:
def qqq(name):
    features = {}
    features['first'] = name[0]
    vowels = ['a', 'e', 'i', 'o', 'u']
    features['vowels'] = len([f for f in name if f in vowels])
    # features['last'] = name[-1]
    return features

qqq("Xaronkn")

{'first': 'X', 'vowels': 2}

In [265]:
def line00000000(name):
    features = {}
    features['last1'] = name[-1:]
    features['last2'] = name[-2:]
    features['last3'] = name[-3:]
    for i in range(len(name) - 1):
    	features[name[i:i+2]] = 1
    	if i + 2 < len(name):
    		features[name[i:i + 3]] = 1
    features['len111'] = len(name)
    return features

line00000000("Xaronkn")

{'Xa': 1,
 'Xar': 1,
 'ar': 1,
 'aro': 1,
 'kn': 1,
 'last1': 'n',
 'last2': 'kn',
 'last3': 'nkn',
 'len111': 7,
 'nk': 1,
 'nkn': 1,
 'on': 1,
 'onk': 1,
 'ro': 1,
 'ron': 1}

In [266]:
def baseline(name):
    features = {}
    features['last'] = name[-1:]
    features['last2'] = name[-2:]
    features['last3'] = name[-3:]
    return features

baisline("Xaronkn")

{'fist': 'X', 'last': 'n'}

In [267]:
def f(name):
    res = dict()
    res['one'] = name[-1]
    res['two'] = name[-2:]
    res['three'] = name[-3:]
    res['1'] = name[0]
    res['2'] = name[0:2]
    return res
f("Xaronkn")

{'1': 'X', '2': 'Xa', 'one': 'n', 'three': 'nkn', 'two': 'kn'}

In [268]:
teams = {
    'arseny' : arseny,
    'line': line,
    'baseline': baseline,
    'line00000000': line00000000,
    'qqq': qqq,
    'f': f
}

In [269]:
competition = []

for team_name, feature_maker in teams.items():
    featuresets = list((feature_maker(name), label) for name, label in names)
    train_set, test_set = featuresets[test_size:], featuresets[:test_size]
    classifier = NaiveBayesClassifier.train(train_set)
    score = accuracy(classifier, test_set)
    competition.append((team_name, score))

In [270]:
classifier.show_most_informative_features(10)

Most Informative Features
                   2last = u'na'          female : male   =     98.4 : 1.0
                   2last = u'la'          female : male   =     73.9 : 1.0
                   2last = u'ia'          female : male   =     40.9 : 1.0
                    last = u'a'           female : male   =     37.4 : 1.0
                   2last = u'sa'          female : male   =     35.0 : 1.0
                    last = u'k'             male : female =     31.7 : 1.0
                   2last = u'us'            male : female =     27.8 : 1.0
                   2last = u'io'            male : female =     27.2 : 1.0
                   2last = u'ra'          female : male   =     27.2 : 1.0
                   2last = u'ta'          female : male   =     26.1 : 1.0


In [271]:
DataFrame.from_records(competition, columns=['Team', 'Score']).sort_values(by="Score", ascending=False)

,Team,Score
2,line00000000,0.834
1,f,0.812
5,arseny,0.806
0,baseline,0.800
4,line,0.778
3,qqq,0.660


In [272]:
def arseny(text):
    features = {}
    features['first'] = text[0]
    features['last'] = text[-1]
    features['2first'] = "".join(text[:2])
    features['2last'] = "".join(text[-2:])
    
    pair_count = 0
    for bigram, counts in Counter(bigrams(text)).items():
        first, second = bigram
        if first == second:
            pair_count += 1
    
    features['pair_count'] = pair_count
    return features

## Sentiment Analysis

In [273]:
from nltk.corpus import movie_reviews

In [274]:
print movie_reviews.readme()

Sentiment Polarity Dataset Version 2.0
Bo Pang and Lillian Lee

http://www.cs.cornell.edu/people/pabo/movie-review-data/

Distributed with NLTK with permission from the authors.


Introduction

This README v2.0 (June, 2004) for the v2.0 polarity dataset comes from
the URL http://www.cs.cornell.edu/people/pabo/movie-review-data .


What's New -- June, 2004

This dataset represents an enhancement of the review corpus v1.0
described in README v1.1: it contains more reviews, and labels were
created with an improved rating-extraction system.


Citation Info 

This data was first used in Bo Pang and Lillian Lee,
``A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization 
Based on Minimum Cuts'',  Proceedings of the ACL, 2004.

@InProceedings{Pang+Lee:04a,
  author =       {Bo Pang and Lillian Lee},
  title =        {A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts},
  booktitle =    "Proceedings of the ACL",
  year =      

In [275]:
movie_reviews.categories()

[u'neg', u'pos']

In [276]:
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
        for fileid in movie_reviews.fileids(category)
]
shuffle(documents)

In [277]:
all_words = FreqDist(w.lower() for w in movie_reviews.words())
# Filter most common
word_features = all_words.keys()[:2000]

In [278]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [279]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [280]:
accuracy(classifier, test_set)

0.62

In [281]:
classifier.show_most_informative_features(5)

Most Informative Features
          contains(sans) = True              neg : pos    =      9.0 : 1.0
    contains(mediocrity) = True              neg : pos    =      7.7 : 1.0
     contains(dismissed) = True              pos : neg    =      7.0 : 1.0
   contains(bruckheimer) = True              neg : pos    =      6.3 : 1.0
        contains(doubts) = True              pos : neg    =      5.8 : 1.0


In [282]:
from nltk import ConfusionMatrix
import numpy as np

In [283]:
classification = classifier.classify_many((feature_set for feature_set, label in featuresets))

In [284]:
confusion_matrix = ConfusionMatrix(list(label for features, label in featuresets), classification)

In [285]:
print confusion_matrix.pretty_format(sort_by_count=True, show_percents=False)
print confusion_matrix.pretty_format(sort_by_count=True, show_percents=True)

    |   n   p |
    |   e   o |
    |   g   s |
----+---------+
neg |<901> 99 |
pos | 234<766>|
----+---------+
(row = reference; col = test)

    |      n      p |
    |      e      o |
    |      g      s |
----+---------------+
neg | <45.0%>  5.0% |
pos |  11.7% <38.3%>|
----+---------------+
(row = reference; col = test)

